<a href="https://colab.research.google.com/github/mobarakol/tutorial_notebooks/blob/main/LLM_LLaMA_QA_LoRA_Working_Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip -q install datasets
!pip -q install sentencepiece
!pip -q install transformers peft bitsandbytes accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 75.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 66.5 MB/s eta 0:00:00


In [3]:
from huggingface_hub import login
# Replace 'your_huggingface_token' with your actual token
HUGGINGFACE_TOKEN = "your token"

# Authenticate with Hugging Face
login(HUGGINGFACE_TOKEN)

print("✅ Successfully logged into Hugging Face!")

✅ Successfully logged into Hugging Face!


#With quantization but without template:

In [4]:
import torch
import os
import random
import numpy as np
from torch.utils.data import DataLoader
from transformers import LlamaForCausalLM, LlamaTokenizer, AdamW
from datasets import Dataset
from peft import get_peft_model, LoraConfig, TaskType
from transformers import AutoTokenizer
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import get_peft_model, LoraConfig, TaskType

# Set random seed for reproducibility
def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(50)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load LLaMA 2.7B tokenizer and model
model_name = "Meta-Llama/Llama-2-7b-chat-hf"

# Load the correct tokenizer for LLaMA-2 using AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)

# Set padding token to EOS if missing
tokenizer.pad_token = tokenizer.eos_token

# Configure 4-bit quantization using bitsandbytes
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",  # Normalized Float 4 (better than standard FP4)
    bnb_4bit_use_double_quant=True,  # Uses secondary quantization for better precision
    bnb_4bit_compute_dtype=torch.float16  # Keeps computation in FP16 for stability
)

# uncomment these first time
# Load LLaMA 2.7B with 4-bit quantization
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    low_cpu_mem_usage=True,
)

base_model.config.pad_token_id = tokenizer.eos_token_id  # Set pad token ID

# Configure LoRA for memory-efficient fine-tuning
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"]  # Apply LoRA to attention layers
)

# Wrap the model with LoRA adapters
model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()  # Verify LoRA trainable parameters

# Define QA training and validation samples
train_qa_samples = [
    {"question": "What is the capital of France?", "answer": "Paris."},
    {"question": "Who wrote '1984'?", "answer": "George Orwell."},
    {"question": "What is the largest planet?", "answer": "Jupiter."},
    {"question": "Who painted the Mona Lisa?", "answer": "Leonardo da Vinci."},
    {"question": "What is the speed of light?", "answer": "299,792 km/s."}
]

valid_qa_samples = [
    {"question": "Which city is the capital of France?", "answer": "Paris."},
    {"question": "Who is the author of '1984'?", "answer": "George Orwell."},
    {"question": "What planet is the biggest?", "answer": "Jupiter."},
    {"question": "Who created the Mona Lisa?", "answer": "Leonardo da Vinci."},
    {"question": "How fast does light travel?", "answer": "299,792 km/s."}
]

# Preprocessing function
def preprocess_data(example):
    input_text = f"Question: {example['question']}\nAnswer: {example['answer']}"
    inputs = tokenizer(input_text, truncation=True, padding="max_length", max_length=256)

    # Copy input_ids to labels
    labels = inputs["input_ids"].copy()

    # Mask question tokens and padding tokens in labels
    question_length = len(tokenizer(f"Question: {example['question']}\nAnswer:")["input_ids"]) - 1
    for i in range(len(labels)):
        if i < question_length or labels[i] == tokenizer.pad_token_id:
            labels[i] = -100  # Ignore these tokens in loss computation

    inputs["labels"] = labels
    return inputs

# Convert datasets and preprocess
dataset_train = Dataset.from_list(train_qa_samples).map(preprocess_data, remove_columns=["question", "answer"])
dataset_valid = Dataset.from_list(valid_qa_samples).map(preprocess_data, remove_columns=["question", "answer"])

# DataLoader Collation
def collate_fn(batch):
    input_ids = torch.tensor([item["input_ids"] for item in batch], dtype=torch.long)
    attention_mask = torch.tensor([item["attention_mask"] for item in batch], dtype=torch.long)
    labels = torch.tensor([item["labels"] for item in batch], dtype=torch.long)
    return input_ids, attention_mask, labels

# Create DataLoaders
batch_size = 2
train_loader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
valid_loader = DataLoader(dataset_valid, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

# Optimizer & Loss Function
optimizer = AdamW(model.parameters(), lr=3e-5)
criterion = torch.nn.CrossEntropyLoss(ignore_index=-100)

# Function to save the best model
def save_best_model(model, tokenizer, epoch, best_loss, current_loss, save_path="./llama-lora-qa-best"):
    if current_loss < best_loss:
        best_loss = current_loss
        os.makedirs(save_path, exist_ok=True)
        model.save_pretrained(save_path)
        tokenizer.save_pretrained(save_path)
        print(f"✅ Best model saved at epoch {epoch} with validation loss: {best_loss:.4f}")
    return best_loss

# Training Function
def train(model, train_loader, valid_loader, optimizer, criterion, num_epochs=3):
    best_val_loss = float("inf")

    for epoch in range(num_epochs):
        model.train()
        total_train_loss = 0

        for batch in train_loader:
            input_ids, attention_mask, labels = [x.to(device) for x in batch]
            optimizer.zero_grad()

            # Forward Pass
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            # Shift logits and labels for loss computation
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()
            shift_logits = shift_logits.view(-1, shift_logits.size(-1))
            shift_labels = shift_labels.view(-1)

            # Compute Loss
            loss = criterion(shift_logits, shift_labels)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()

        avg_train_loss = total_train_loss / len(train_loader)
        avg_val_loss = validate(model, valid_loader, criterion)

        print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

        # Save best model
        best_val_loss = save_best_model(model, tokenizer, epoch + 1, best_val_loss, avg_val_loss)

# Validation Function
def validate(model, dataloader, criterion):
    model.eval()
    total_loss = 0

    with torch.no_grad():
        for batch in dataloader:
            input_ids, attention_mask, labels = [x.to(device) for x in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()
            shift_logits = shift_logits.view(-1, shift_logits.size(-1))
            shift_labels = shift_labels.view(-1)

            loss = criterion(shift_logits, shift_labels)
            total_loss += loss.item()

    return total_loss / len(dataloader)

# Start Training
train(model, train_loader, valid_loader, optimizer, criterion, num_epochs=20)


import torch
from transformers import LlamaForCausalLM, LlamaTokenizer
from peft import PeftModel

# Path to the best fine-tuned model
best_model_path = "./llama-lora-qa-best"


# Load the LoRA fine-tuned adapter
model = PeftModel.from_pretrained(base_model, best_model_path).to("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained(best_model_path, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token  # Ensure correct padding

# Function to generate answers
def generate_answer(question):
    model.eval()
    input_text = f"Question: {question}\nAnswer:"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=256).to(model.device)

    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=50, pad_token_id=tokenizer.eos_token_id)

    answer = tokenizer.decode(output[0], skip_special_tokens=True).split("Answer:")[-1].strip()
    return answer

# Example inference
question = "Provide only short answer of the question: Who created the Mona Lisa?"
answer = generate_answer(question)
print(f"Q: {question}\nA: {answer}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

...

trainable params: 8,388,608 || all params: 6,746,804,224 || trainable%: 0.1243


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/20, Train Loss: 1.3395, Val Loss: 1.1618
✅ Best model saved at epoch 1 with validation loss: 1.1618
Epoch 2/20, Train Loss: 0.9050, Val Loss: 0.9045
✅ Best model saved at epoch 2 with validation loss: 0.9045
Epoch 3/20, Train Loss: 0.9064, Val Loss: 0.6462
✅ Best model saved at epoch 3 with validation loss: 0.6462
Epoch 4/20, Train Loss: 0.5318, Val Loss: 0.4623
✅ Best model saved at epoch 4 with validation loss: 0.4623
Epoch 5/20, Train Loss: 0.2958, Val Loss: 0.3614
✅ Best model saved at epoch 5 with validation loss: 0.3614
Epoch 6/20, Train Loss: 0.2159, Val Loss: 0.2995
✅ Best model saved at epoch 6 with validation loss: 0.2995
Epoch 7/20, Train Loss: 0.2165, Val Loss: 0.2550
✅ Best model saved at epoch 7 with validation loss: 0.2550
Epoch 8/20, Train Loss: 0.1603, Val Loss: 0.2218
✅ Best model saved at epoch 8 with validation loss: 0.2218
Epoch 9/20, Train Loss: 0.1552, Val Loss: 0.1919
✅ Best model saved at epoch 9 with validation loss: 0.1919
Epoch 10/20, Train Loss: 0.1

#With quantization and Pytorch Dataset class but without Template

In [6]:
import torch
import os
import random
import numpy as np
from torch.utils.data import Dataset, DataLoader
from transformers import LlamaForCausalLM, LlamaTokenizer, AdamW, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import get_peft_model, LoraConfig, TaskType, PeftModel

# Set random seed for reproducibility
def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(50)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load LLaMA 2.7B tokenizer and model
model_name = "Meta-Llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token

# Configure 4-bit quantization using bitsandbytes
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16
)

# Load LLaMA 2.7B with 4-bit quantization
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    low_cpu_mem_usage=True,
)
base_model.config.pad_token_id = tokenizer.eos_token_id

# Configure LoRA
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"]
)

# Wrap the model with LoRA adapters
model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()

# Define a custom PyTorch Dataset class
class QADataset(Dataset):
    def __init__(self, qa_samples, tokenizer, max_length=256):
        self.qa_samples = qa_samples
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.qa_samples)

    def __getitem__(self, idx):
        example = self.qa_samples[idx]
        formatted_data = [
            {"role": "system", "content": [{"type": "text", "text": "Provide only short answer of the question:"}]},
            {"role": "user", "content": [{"type": "text", "text": example['question']}]},
            {"role": "assistant", "content": [{"type": "text", "text": example['answer']}]}
        ]
        input_text = tokenizer.apply_chat_template(formatted_data, tokenize=False)
        inputs = self.tokenizer(input_text, truncation=True, padding="max_length", max_length=self.max_length, return_tensors="pt")

        labels = inputs["input_ids"].clone()
        question_length = len(self.tokenizer.apply_chat_template(formatted_data[:-1], tokenize=False))
        labels[:, :question_length] = -100  # Mask question tokens
        labels[labels == self.tokenizer.pad_token_id] = -100  # Mask padding tokens

        return {"input_ids": inputs["input_ids"].squeeze(0), "attention_mask": inputs["attention_mask"].squeeze(0), "labels": labels.squeeze(0)}

# Define training and validation datasets
train_qa_samples = [
    {"question": "What is the capital of France?", "answer": "Paris."},
    {"question": "Who wrote '1984'?", "answer": "George Orwell."},
    {"question": "What is the largest planet?", "answer": "Jupiter."},
    {"question": "Who painted the Mona Lisa?", "answer": "Leonardo da Vinci."},
    {"question": "What is the speed of light?", "answer": "299,792 km/s."}
]
valid_qa_samples = [
    {"question": "Which city is the capital of France?", "answer": "Paris."},
    {"question": "Who is the author of '1984'?", "answer": "George Orwell."},
    {"question": "What planet is the biggest?", "answer": "Jupiter."},
    {"question": "Who created the Mona Lisa?", "answer": "Leonardo da Vinci."},
    {"question": "How fast does light travel?", "answer": "299,792 km/s."}
]

# Create dataset instances
dataset_train = QADataset(train_qa_samples, tokenizer)
dataset_valid = QADataset(valid_qa_samples, tokenizer)

# Define DataLoader
def collate_fn(batch):
    input_ids = torch.stack([item["input_ids"] for item in batch])
    attention_mask = torch.stack([item["attention_mask"] for item in batch])
    labels = torch.stack([item["labels"] for item in batch])
    return input_ids, attention_mask, labels

batch_size = 2
train_loader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
valid_loader = DataLoader(dataset_valid, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

# Optimizer & Loss Function
optimizer = AdamW(model.parameters(), lr=3e-5)
criterion = torch.nn.CrossEntropyLoss(ignore_index=-100)

# Function to save the best model
def save_best_model(model, tokenizer, epoch, best_loss, current_loss, save_path="./llama-lora-qa-best-dataset"):
    if current_loss < best_loss:
        best_loss = current_loss
        os.makedirs(save_path, exist_ok=True)
        model.save_pretrained(save_path)
        tokenizer.save_pretrained(save_path)
        print(f"✅ Best model saved at epoch {epoch} with validation loss: {best_loss:.4f}")
    return best_loss

# Training Function
def train(model, train_loader, valid_loader, optimizer, criterion, num_epochs=3, save_path="./llama-lora-qa-best-dataset"):
    best_val_loss = float("inf")
    for epoch in range(num_epochs):
        model.train()
        total_train_loss = 0
        for batch in train_loader:
            input_ids, attention_mask, labels = [x.to(device) for x in batch]
            optimizer.zero_grad()
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous().view(-1)
            shift_logits = shift_logits.view(-1, shift_logits.size(-1))
            loss = criterion(shift_logits, shift_labels)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()

        avg_train_loss = total_train_loss / len(train_loader)
        avg_val_loss = validate(model, valid_loader, criterion)

        print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

        # Save best model
        best_val_loss = save_best_model(model, tokenizer, epoch + 1, best_val_loss, avg_val_loss, save_path=save_path)


# Validation Function
def validate(model, dataloader, criterion):
    model.eval()
    total_loss = 0

    with torch.no_grad():
        for batch in dataloader:
            input_ids, attention_mask, labels = [x.to(device) for x in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()
            shift_logits = shift_logits.view(-1, shift_logits.size(-1))
            shift_labels = shift_labels.view(-1)

            loss = criterion(shift_logits, shift_labels)
            total_loss += loss.item()

    return total_loss / len(dataloader)

# Path to the best fine-tuned model
best_model_path = "./llama-lora-qa-best-dataset"

# Start Training
train(model, train_loader, valid_loader, optimizer, criterion, num_epochs=20, save_path=best_model_path)

import torch
from transformers import LlamaForCausalLM, LlamaTokenizer
from peft import PeftModel


# Load the LoRA fine-tuned adapter
model = PeftModel.from_pretrained(base_model, best_model_path).to("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained(best_model_path, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token  # Ensure correct padding

# Function to generate answers
def generate_answer(question):
    model.eval()
    input_text = f"Question: {question}\nAnswer:"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=256).to(model.device)

    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=50, pad_token_id=tokenizer.eos_token_id)

    answer = tokenizer.decode(output[0], skip_special_tokens=True).split("Answer:")[-1].strip()
    return answer

# Example inference
question = "Provide only short answer of the question: Who created the Mona Lisa?"
answer = generate_answer(question)
print(f"Q: {question}\nA: {answer}")

trainable params: 8,388,608 || all params: 6,746,804,224 || trainable%: 0.1243
Epoch 1/20, Train Loss: 1.3395, Val Loss: 1.1618
✅ Best model saved at epoch 1 with validation loss: 1.1618
Epoch 2/20, Train Loss: 0.9050, Val Loss: 0.9045
✅ Best model saved at epoch 2 with validation loss: 0.9045
Epoch 3/20, Train Loss: 0.9064, Val Loss: 0.6462
✅ Best model saved at epoch 3 with validation loss: 0.6462
Epoch 4/20, Train Loss: 0.5318, Val Loss: 0.4623
✅ Best model saved at epoch 4 with validation loss: 0.4623
Epoch 5/20, Train Loss: 0.2958, Val Loss: 0.3614
✅ Best model saved at epoch 5 with validation loss: 0.3614
Epoch 6/20, Train Loss: 0.2159, Val Loss: 0.2995
✅ Best model saved at epoch 6 with validation loss: 0.2995
Epoch 7/20, Train Loss: 0.2165, Val Loss: 0.2550
✅ Best model saved at epoch 7 with validation loss: 0.2550
Epoch 8/20, Train Loss: 0.1603, Val Loss: 0.2218
✅ Best model saved at epoch 8 with validation loss: 0.2218
Epoch 9/20, Train Loss: 0.1552, Val Loss: 0.1919
✅ Best m

#With quantization, Pytorch Dataset class, and QA Template

In [62]:
import torch
import os
import random
import copy
import numpy as np
from torch.utils.data import Dataset, DataLoader
from transformers import (
    LlamaForCausalLM,
    LlamaTokenizer,
    AdamW,
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig
)
from peft import get_peft_model, LoraConfig, TaskType, PeftModel


# Set random seed for reproducibility
def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(50)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load LLaMA 2.7B tokenizer and model
model_name = "Meta-Llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token

# Configure 4-bit quantization using bitsandbytes
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16  # or torch.bfloat16 if you see NaN issues
)

# Load LLaMA 2.7B with 4-bit quantization
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    low_cpu_mem_usage=True,
)
base_model.config.pad_token_id = tokenizer.eos_token_id

# Configure LoRA
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"]
)

# Wrap the model with LoRA adapters
model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()

# -------------- 2) A custom PyTorch Dataset class using format_data --------------
def format_data(sample):
    """
    Flatten content fields so they're simple strings.
    """
    return [
        {
            "role": "system",
            "content": "Provide only short answer of the question:"
        },
        {
            "role": "user",
            "content": sample["question"]
        },
        {
            "role": "assistant",
            "content": sample["answer"]
        }
    ]

class QADataset(Dataset):
    def __init__(self, qa_samples, tokenizer, max_length=256):
        self.qa_samples = qa_samples
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.qa_samples)

    def __getitem__(self, idx):
        sample = self.qa_samples[idx]
        # 1) Format as simple strings
        messages = format_data(sample)

        # 2) Use apply_chat_template
        input_text = self.tokenizer.apply_chat_template(messages, tokenize=False)

        # 3) Tokenize
        inputs = self.tokenizer(
            input_text,
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt"
        )

        labels = inputs["input_ids"].clone()

        #system_question masking
        system_question_msg = copy.deepcopy(messages)
        system_question_msg[-1]["content"] = ""  # Remove the assistant text
        system_question_text = self.tokenizer.apply_chat_template(system_question_msg, tokenize=False)
        system_question_len = len(self.tokenizer(system_question_text, add_special_tokens=False)["input_ids"]) - 1
        # print('mobarakkkkkkkk:', labels.shape, system_question_len)
        labels[:,:system_question_len] = -100  # Mask system msg + question tokens

        # example: only mask padding
        labels[labels == self.tokenizer.pad_token_id] = -100

        return {
            "input_ids": inputs["input_ids"].squeeze(0),
            "attention_mask": inputs["attention_mask"].squeeze(0),
            "labels": labels.squeeze(0),
        }


# Define training and validation datasets
train_qa_samples = [
    {"question": "What is the capital of France?", "answer": "Paris."},
    {"question": "Who wrote '1984'?", "answer": "George Orwell."},
    {"question": "What is the largest planet?", "answer": "Jupiter."},
    {"question": "Who painted the Mona Lisa?", "answer": "Leonardo da Vinci."},
    {"question": "What is the speed of light?", "answer": "299,792 km/s."}
]
valid_qa_samples = [
    {"question": "Which city is the capital of France?", "answer": "Paris."},
    {"question": "Who is the author of '1984'?", "answer": "George Orwell."},
    {"question": "What planet is the biggest?", "answer": "Jupiter."},
    {"question": "Who created the Mona Lisa?", "answer": "Leonardo da Vinci."},
    {"question": "How fast does light travel?", "answer": "299,792 km/s."}
]

# Create dataset instances
dataset_train = QADataset(train_qa_samples, tokenizer)
dataset_valid = QADataset(valid_qa_samples, tokenizer)

def collate_fn(batch):
    input_ids = torch.stack([item["input_ids"] for item in batch])
    attention_mask = torch.stack([item["attention_mask"] for item in batch])
    labels = torch.stack([item["labels"] for item in batch])
    return input_ids, attention_mask, labels

batch_size = 2
train_loader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
valid_loader = DataLoader(dataset_valid, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

# Optimizer & Loss Function
optimizer = AdamW(model.parameters(), lr=3e-5)
criterion = torch.nn.CrossEntropyLoss(ignore_index=-100)

# Function to save the best model
def save_best_model(model, tokenizer, epoch, best_loss, current_loss, save_path="./llama-lora-qa-best-dataset"):
    if current_loss < best_loss:
        best_loss = current_loss
        os.makedirs(save_path, exist_ok=True)
        model.save_pretrained(save_path)
        tokenizer.save_pretrained(save_path)
        print(f"✅ Best model saved at epoch {epoch} with validation loss: {best_loss:.4f}")
    return best_loss

# Training Function
def train(model, train_loader, valid_loader, optimizer, criterion, num_epochs=3, save_path="./llama-lora-qa-best-dataset"):
    best_val_loss = float("inf")
    for epoch in range(num_epochs):
        model.train()
        total_train_loss = 0
        for batch in train_loader:
            input_ids, attention_mask, labels = [x.to(device) for x in batch]
            optimizer.zero_grad()
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            # Shift for next-token prediction
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous().view(-1)
            shift_logits = shift_logits.view(-1, shift_logits.size(-1))

            loss = criterion(shift_logits, shift_labels)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()

        avg_train_loss = total_train_loss / len(train_loader)
        avg_val_loss = validate(model, valid_loader, criterion)

        print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

        # Save best model
        best_val_loss = save_best_model(model, tokenizer, epoch + 1, best_val_loss, avg_val_loss, save_path=save_path)

# Validation Function
def validate(model, dataloader, criterion):
    model.eval()
    total_loss = 0

    with torch.no_grad():
        for batch in dataloader:
            input_ids, attention_mask, labels = [x.to(device) for x in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous().view(-1)
            shift_logits = shift_logits.view(-1, shift_logits.size(-1))

            loss = criterion(shift_logits, shift_labels)
            total_loss += loss.item()

    return total_loss / len(dataloader)

# Start Training
best_model_path = "./llama-lora-qa-best-dataset-template"
train(model, train_loader, valid_loader, optimizer, criterion, num_epochs=20, save_path=best_model_path)

# ----------------------------------------------------------------------------
# Inference Section (Load best model & generate short answer)
# ----------------------------------------------------------------------------
import torch
from transformers import LlamaForCausalLM, LlamaTokenizer
from peft import PeftModel

# Load the LoRA fine-tuned adapter
model = PeftModel.from_pretrained(base_model, best_model_path).to(device)
tokenizer = AutoTokenizer.from_pretrained(best_model_path, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token  # Ensure correct padding

# Function to generate answers
def generate_answer(question):
    model.eval()
    input_text = f"Question: {question}\nAnswer:"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=256).to(device)

    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=50, pad_token_id=tokenizer.eos_token_id)

    answer = tokenizer.decode(output[0], skip_special_tokens=True).split("Answer:")[-1].strip()
    return answer

# Example inference
question = "Provide only short answer of the question: Who created the Mona Lisa?"
answer = generate_answer(question)
print(f"Q: {question}\nA: {answer}")


trainable params: 8,388,608 || all params: 6,746,804,224 || trainable%: 0.1243


/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/20, Train Loss: 6.4375, Val Loss: 6.1953
✅ Best model saved at epoch 1 with validation loss: 6.1953
Epoch 2/20, Train Loss: 5.8281, Val Loss: 5.6165
✅ Best model saved at epoch 2 with validation loss: 5.6165
Epoch 3/20, Train Loss: 6.8561, Val Loss: 4.9036
✅ Best model saved at epoch 3 with validation loss: 4.9036
Epoch 4/20, Train Loss: 4.4661, Val Loss: 4.0781
✅ Best model saved at epoch 4 with validation loss: 4.0781
Epoch 5/20, Train Loss: 4.1562, Val Loss: 3.1504
✅ Best model saved at epoch 5 with validation loss: 3.1504
Epoch 6/20, Train Loss: 3.0430, Val Loss: 2.1048
✅ Best model saved at epoch 6 with validation loss: 2.1048
Epoch 7/20, Train Loss: 1.4263, Val Loss: 1.1530
✅ Best model saved at epoch 7 with validation loss: 1.1530
Epoch 8/20, Train Loss: 0.7558, Val Loss: 0.7035
✅ Best model saved at epoch 8 with validation loss: 0.7035
Epoch 9/20, Train Loss: 0.6085, Val Loss: 0.5944
✅ Best model saved at epoch 9 with validation loss: 0.5944
Epoch 10/20, Train Loss: 0.4

In [63]:
def generate_answer(question):
    """
    Generate an answer using the same chat template we used in training.
    """
    model.eval()

    # 1) Build messages with an empty assistant content.
    #    This ensures we keep the same format as in training.
    messages = [
        {
            "role": "system",
            "content": "Provide only short answer of the question:"
        },
        {
            "role": "user",
            "content": question
        },
        {
            "role": "assistant",
            "content": ""  # We leave this empty for generation
        }
    ]

    # 2) Convert messages to text via apply_chat_template
    input_text = tokenizer.apply_chat_template(messages, tokenize=False)

    # 3) Tokenize the chat
    inputs = tokenizer(
        input_text,
        return_tensors="pt",
        truncation=True,
        max_length=256
    ).to(device)

    # 4) Generate tokens
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=50,
            pad_token_id=tokenizer.eos_token_id
        )

    # Trim the generated ids to remove the input ids
    trimmed_output = [
        out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, output)
    ]
    answer = tokenizer.decode(trimmed_output[0], skip_special_tokens=True).strip()

    return answer

model = PeftModel.from_pretrained(base_model, best_model_path).to(device)
tokenizer = AutoTokenizer.from_pretrained(best_model_path, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token  # Ensure correct padding

# Example inference
question = "What is the capital of France?"
answer = generate_answer(question)
print(f"Q: {question}\nA: {answer}")


Q: What is the capital of France?
A: : Paris.


LLaMA 3

In [ ]:
import torch
import os
import random
import numpy as np
from torch.utils.data import DataLoader
from transformers import LlamaForCausalLM, LlamaTokenizer, AdamW
from datasets import Dataset
from peft import get_peft_model, LoraConfig, TaskType
from transformers import AutoTokenizer
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import get_peft_model, LoraConfig, TaskType

# Set random seed for reproducibility
def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(50)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load LLaMA 2.7B tokenizer and model
# model_name = "Meta-Llama/Llama-2-7b-chat-hf"
model_name = "meta-llama/Llama-3.2-1B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
# tokenizer = LlamaTokenizer.from_pretrained(model_name)

# Load the correct tokenizer for LLaMA-2 using AutoTokenizer
# tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)

# Set padding token to EOS if missing
tokenizer.pad_token = tokenizer.eos_token

# base_model = LlamaForCausalLM.from_pretrained(
#     model_name, torch_dtype=torch.float16, device_map="auto"
# )

# Configure 4-bit quantization using bitsandbytes
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",  # Normalized Float 4 (better than standard FP4)
    bnb_4bit_use_double_quant=True,  # Uses secondary quantization for better precision
    bnb_4bit_compute_dtype=torch.float16  # Keeps computation in FP16 for stability
)

# Load LLaMA 2.7B with 4-bit quantization
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

base_model.config.pad_token_id = tokenizer.eos_token_id  # Set pad token ID

# Configure LoRA for memory-efficient fine-tuning
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"]  # Apply LoRA to attention layers
)

# Wrap the model with LoRA adapters
model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()  # Verify LoRA trainable parameters

# Define QA training and validation samples
train_qa_samples = [
    {"question": "What is the capital of France?", "answer": "Paris."},
    {"question": "Who wrote '1984'?", "answer": "George Orwell."},
    {"question": "What is the largest planet?", "answer": "Jupiter."},
    {"question": "Who painted the Mona Lisa?", "answer": "Leonardo da Vinci."},
    {"question": "What is the speed of light?", "answer": "299,792 km/s."}
]

valid_qa_samples = [
    {"question": "Which city is the capital of France?", "answer": "Paris."},
    {"question": "Who is the author of '1984'?", "answer": "George Orwell."},
    {"question": "What planet is the biggest?", "answer": "Jupiter."},
    {"question": "Who created the Mona Lisa?", "answer": "Leonardo da Vinci."},
    {"question": "How fast does light travel?", "answer": "299,792 km/s."}
]

# Preprocessing function
def preprocess_data(example):
    input_text = f"Question: {example['question']}\nAnswer: {example['answer']}"
    inputs = tokenizer(input_text, truncation=True, padding="max_length", max_length=256)

    # Copy input_ids to labels
    labels = inputs["input_ids"].copy()

    # Mask question tokens and padding tokens in labels
    question_length = len(tokenizer(f"Question: {example['question']}\nAnswer:")["input_ids"]) - 1
    for i in range(len(labels)):
        if i < question_length or labels[i] == tokenizer.pad_token_id:
            labels[i] = -100  # Ignore these tokens in loss computation

    inputs["labels"] = labels
    return inputs

# Convert datasets and preprocess
dataset_train = Dataset.from_list(train_qa_samples).map(preprocess_data, remove_columns=["question", "answer"])
dataset_valid = Dataset.from_list(valid_qa_samples).map(preprocess_data, remove_columns=["question", "answer"])

# DataLoader Collation
def collate_fn(batch):
    input_ids = torch.tensor([item["input_ids"] for item in batch], dtype=torch.long)
    attention_mask = torch.tensor([item["attention_mask"] for item in batch], dtype=torch.long)
    labels = torch.tensor([item["labels"] for item in batch], dtype=torch.long)
    return input_ids, attention_mask, labels

# Create DataLoaders
batch_size = 2
train_loader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
valid_loader = DataLoader(dataset_valid, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

# Optimizer & Loss Function
optimizer = AdamW(model.parameters(), lr=3e-5)
criterion = torch.nn.CrossEntropyLoss(ignore_index=-100)

# Function to save the best model
def save_best_model(model, tokenizer, epoch, best_loss, current_loss, save_path="./llama-lora-qa-best"):
    if current_loss < best_loss:
        best_loss = current_loss
        os.makedirs(save_path, exist_ok=True)
        model.save_pretrained(save_path)
        tokenizer.save_pretrained(save_path)
        print(f"✅ Best model saved at epoch {epoch} with validation loss: {best_loss:.4f}")
    return best_loss

# Training Function
def train(model, train_loader, valid_loader, optimizer, criterion, num_epochs=3):
    best_val_loss = float("inf")

    for epoch in range(num_epochs):
        model.train()
        total_train_loss = 0

        for batch in train_loader:
            input_ids, attention_mask, labels = [x.to(device) for x in batch]
            optimizer.zero_grad()

            # Forward Pass
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            # Shift logits and labels for loss computation
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()
            shift_logits = shift_logits.view(-1, shift_logits.size(-1))
            shift_labels = shift_labels.view(-1)

            # Compute Loss
            loss = criterion(shift_logits, shift_labels)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()

        avg_train_loss = total_train_loss / len(train_loader)
        avg_val_loss = validate(model, valid_loader, criterion)

        print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

        # Save best model
        best_val_loss = save_best_model(model, tokenizer, epoch + 1, best_val_loss, avg_val_loss)

# Validation Function
def validate(model, dataloader, criterion):
    model.eval()
    total_loss = 0

    with torch.no_grad():
        for batch in dataloader:
            input_ids, attention_mask, labels = [x.to(device) for x in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()
            shift_logits = shift_logits.view(-1, shift_logits.size(-1))
            shift_labels = shift_labels.view(-1)

            loss = criterion(shift_logits, shift_labels)
            total_loss += loss.item()

    return total_loss / len(dataloader)

# Start Training
train(model, train_loader, valid_loader, optimizer, criterion, num_epochs=20)


import torch
from transformers import LlamaForCausalLM, LlamaTokenizer
from peft import PeftModel

# Path to the best fine-tuned model
best_model_path = "./llama-lora-qa-best"

# Load the base LLaMA 2.7B model
# base_model = LlamaForCausalLM.from_pretrained(
#     "Meta-Llama/Llama-2-7b-chat-hf",
#     torch_dtype=torch.float16,
#     device_map="auto"
# )

# base_model = AutoModelForCausalLM.from_pretrained(
#     "meta-llama/Llama-2-7b-chat-hf",
#     quantization_config=bnb_config,
#     device_map="auto"
# )

# Load the LoRA fine-tuned adapter
model = PeftModel.from_pretrained(base_model, best_model_path).to("cuda" if torch.cuda.is_available() else "cpu")

# Load the fine-tuned tokenizer
# tokenizer = LlamaTokenizer.from_pretrained(best_model_path)
tokenizer = AutoTokenizer.from_pretrained(best_model_path, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token  # Ensure correct padding

# Function to generate answers
def generate_answer(question):
    model.eval()
    input_text = f"Question: {question}\nAnswer:"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=256).to(model.device)

    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=50, pad_token_id=tokenizer.eos_token_id)

    answer = tokenizer.decode(output[0], skip_special_tokens=True).split("Answer:")[-1].strip()
    return answer

# Example inference
question = "What is the capital of France?"
answer = generate_answer(question)
print(f"Q: {question}\nA: {answer}")

trainable params: 1,703,936 || all params: 1,237,518,336 || trainable%: 0.1377


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Epoch 1/20, Train Loss: 0.8744, Val Loss: 0.9499
✅ Best model saved at epoch 1 with validation loss: 0.9499
Epoch 2/20, Train Loss: 0.8475, Val Loss: 0.8506
✅ Best model saved at epoch 2 with validation loss: 0.8506
Epoch 3/20, Train Loss: 0.6850, Val Loss: 0.7530
✅ Best model saved at epoch 3 with validation loss: 0.7530
Epoch 4/20, Train Loss: 0.4630, Val Loss: 0.6541
✅ Best model saved at epoch 4 with validation loss: 0.6541
Epoch 5/20, Train Loss: 0.3314, Val Loss: 0.5708
✅ Best model saved at epoch 5 with validation loss: 0.5708
Epoch 6/20, Train Loss: 0.3336, Val Loss: 0.4996
✅ Best model saved at epoch 6 with validation loss: 0.4996
Epoch 7/20, Train Loss: 0.2611, Val Loss: 0.4373
✅ Best model saved at epoch 7 with validation loss: 0.4373
Epoch 8/20, Train Loss: 0.1585, Val Loss: 0.3819
✅ Best model saved at epoch 8 with validation loss: 0.3819
Epoch 9/20, Train Loss: 0.1165, Val Loss: 0.3395
✅ Best model saved at epoch 9 with validation loss: 0.3395
Epoch 10/20, Train Loss: 0.1

In [ ]:
# Function to generate answers
def generate_answer(question):
    model.eval()
    input_text = f"Question: {question}\nAnswer:"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=256).to(model.device)

    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=50, pad_token_id=tokenizer.eos_token_id)

    answer = tokenizer.decode(output[0], skip_special_tokens=True).split("Answer:")[-1].strip()
    return answer

# Example inference
question = "Provide only short answer of the question: Who created the Mona Lisa?"
answer = generate_answer(question)
print(f"Q: {question}\nA: {answer}")

Q: Provide only short answer of the question: Who created the Mona Lisa?
A: Leonardo da Vinci.


In [ ]:
import transformers

print(transformers.__version__)

4.47.1


In [ ]:
import bitsandbytes

print(bitsandbytes.__version__)

0.45.1


In [ ]:
from huggingface_hub import login
# Replace 'your_huggingface_token' with your actual token
HUGGINGFACE_TOKEN = "your token"

# Authenticate with Hugging Face
login(HUGGINGFACE_TOKEN)

print("✅ Successfully logged into Hugging Face!")

✅ Successfully logged into Hugging Face!


Quantized Llama 2.7 chat LoRA finetuning with batch greedy search:

In [ ]:
import torch
import os
import random
import numpy as np
from torch.utils.data import DataLoader
from transformers import LlamaForCausalLM, LlamaTokenizer, AdamW
from datasets import Dataset
from peft import get_peft_model, LoraConfig, TaskType
from transformers import AutoTokenizer
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import get_peft_model, LoraConfig, TaskType

# Set random seed for reproducibility
def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(50)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load LLaMA 2.7B tokenizer and model
model_name = "Meta-Llama/Llama-2-7b-chat-hf"
# tokenizer = LlamaTokenizer.from_pretrained(model_name)
# tokenizer.pad_token = tokenizer.eos_token  # Set padding token

# Load the correct tokenizer for LLaMA-2 using AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)

# Set padding token to EOS if missing
tokenizer.pad_token = tokenizer.eos_token

# base_model = LlamaForCausalLM.from_pretrained(
#     model_name, torch_dtype=torch.float16, device_map="auto"
# )

# Configure 4-bit quantization using bitsandbytes
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",  # Normalized Float 4 (better than standard FP4)
    bnb_4bit_use_double_quant=True,  # Uses secondary quantization for better precision
    bnb_4bit_compute_dtype=torch.float16  # Keeps computation in FP16 for stability
)

# uncomment these first time
# Load LLaMA 2.7B with 4-bit quantization
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

base_model.config.pad_token_id = tokenizer.eos_token_id  # Set pad token ID

# Configure LoRA for memory-efficient fine-tuning
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"]  # Apply LoRA to attention layers
)

# Wrap the model with LoRA adapters
model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()  # Verify LoRA trainable parameters

# Define QA training and validation samples
train_qa_samples = [
    {"question": "What is the capital of France?", "answer": "Paris."},
    {"question": "Who wrote '1984'?", "answer": "George Orwell."},
    {"question": "What is the largest planet?", "answer": "Jupiter."},
    {"question": "Who painted the Mona Lisa?", "answer": "Leonardo da Vinci."},
    {"question": "What is the speed of light?", "answer": "299,792 km/s."}
]

valid_qa_samples = [
    {"question": "Which city is the capital of France?", "answer": "Paris."},
    {"question": "Who is the author of '1984'?", "answer": "George Orwell."},
    {"question": "What planet is the biggest?", "answer": "Jupiter."},
    {"question": "Who created the Mona Lisa?", "answer": "Leonardo da Vinci."},
    {"question": "How fast does light travel?", "answer": "299,792 km/s."}
]

# Preprocessing function
def preprocess_data(example):
    input_text = f"Question: {example['question']}\nAnswer: {example['answer']}"
    inputs = tokenizer(input_text, truncation=True, padding="max_length", max_length=256)

    # Copy input_ids to labels
    labels = inputs["input_ids"].copy()

    # Mask question tokens and padding tokens in labels
    question_length = len(tokenizer(f"Question: {example['question']}\nAnswer:")["input_ids"]) - 1
    for i in range(len(labels)):
        if i < question_length or labels[i] == tokenizer.pad_token_id:
            labels[i] = -100  # Ignore these tokens in loss computation

    inputs["labels"] = labels
    return inputs

# Convert datasets and preprocess
dataset_train = Dataset.from_list(train_qa_samples).map(preprocess_data, remove_columns=["question", "answer"])
dataset_valid = Dataset.from_list(valid_qa_samples).map(preprocess_data, remove_columns=["question", "answer"])

# DataLoader Collation
def collate_fn(batch):
    input_ids = torch.tensor([item["input_ids"] for item in batch], dtype=torch.long)
    attention_mask = torch.tensor([item["attention_mask"] for item in batch], dtype=torch.long)
    labels = torch.tensor([item["labels"] for item in batch], dtype=torch.long)
    return input_ids, attention_mask, labels

# Create DataLoaders
batch_size = 2
train_loader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
valid_loader = DataLoader(dataset_valid, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

# Optimizer & Loss Function
optimizer = AdamW(model.parameters(), lr=3e-5)
criterion = torch.nn.CrossEntropyLoss(ignore_index=-100)

# Function to save the best model
def save_best_model(model, tokenizer, epoch, best_loss, current_loss, save_path="./llama-lora-qa-best"):
    if current_loss < best_loss:
        best_loss = current_loss
        os.makedirs(save_path, exist_ok=True)
        model.save_pretrained(save_path)
        tokenizer.save_pretrained(save_path)
        print(f"✅ Best model saved at epoch {epoch} with validation loss: {best_loss:.4f}")
    return best_loss

# Training Function
def train(model, train_loader, valid_loader, optimizer, criterion, num_epochs=3):
    best_val_loss = float("inf")

    for epoch in range(num_epochs):
        model.train()
        total_train_loss = 0

        for batch in train_loader:
            input_ids, attention_mask, labels = [x.to(device) for x in batch]
            optimizer.zero_grad()

            # Forward Pass
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            # Shift logits and labels for loss computation
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()
            shift_logits = shift_logits.view(-1, shift_logits.size(-1))
            shift_labels = shift_labels.view(-1)

            # Compute Loss
            loss = criterion(shift_logits, shift_labels)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()

        avg_train_loss = total_train_loss / len(train_loader)
        avg_val_loss = validate(model, valid_loader, criterion)

        print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

        # Save best model
        best_val_loss = save_best_model(model, tokenizer, epoch + 1, best_val_loss, avg_val_loss)

# Validation Function
def validate(model, dataloader, criterion):
    model.eval()
    total_loss = 0

    with torch.no_grad():
        for batch in dataloader:
            input_ids, attention_mask, labels = [x.to(device) for x in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()
            shift_logits = shift_logits.view(-1, shift_logits.size(-1))
            shift_labels = shift_labels.view(-1)

            loss = criterion(shift_logits, shift_labels)
            total_loss += loss.item()

    return total_loss / len(dataloader)

# Start Training
train(model, train_loader, valid_loader, optimizer, criterion, num_epochs=20)


import torch
from transformers import LlamaForCausalLM, LlamaTokenizer
from peft import PeftModel

# Path to the best fine-tuned model
best_model_path = "./llama-lora-qa-best"

# Load the LoRA fine-tuned adapter
model = PeftModel.from_pretrained(base_model, best_model_path).to("cuda" if torch.cuda.is_available() else "cpu")

# Load the fine-tuned tokenizer
# tokenizer = LlamaTokenizer.from_pretrained(best_model_path)
tokenizer = AutoTokenizer.from_pretrained(best_model_path, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token  # Ensure correct padding

# Function to generate answers
def generate_answer(question):
    model.eval()
    input_text = f"Question: {question}\nAnswer:"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=256).to(model.device)

    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=50, pad_token_id=tokenizer.eos_token_id)

    answer = tokenizer.decode(output[0], skip_special_tokens=True).split("Answer:")[-1].strip()
    return answer

# Example inference
question = "Provide only short answer of the question: Who created the Mona Lisa?"
answer = generate_answer(question)
print(f"Q: {question}\nA: {answer}")

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

...

trainable params: 8,388,608 || all params: 6,746,804,224 || trainable%: 0.1243


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/20, Train Loss: 1.3395, Val Loss: 1.1618
✅ Best model saved at epoch 1 with validation loss: 1.1618
Epoch 2/20, Train Loss: 0.9050, Val Loss: 0.9045
✅ Best model saved at epoch 2 with validation loss: 0.9045
Epoch 3/20, Train Loss: 0.9064, Val Loss: 0.6462
✅ Best model saved at epoch 3 with validation loss: 0.6462
Epoch 4/20, Train Loss: 0.5318, Val Loss: 0.4623
✅ Best model saved at epoch 4 with validation loss: 0.4623
Epoch 5/20, Train Loss: 0.2958, Val Loss: 0.3614
✅ Best model saved at epoch 5 with validation loss: 0.3614
Epoch 6/20, Train Loss: 0.2159, Val Loss: 0.2995
✅ Best model saved at epoch 6 with validation loss: 0.2995
Epoch 7/20, Train Loss: 0.2165, Val Loss: 0.2550
✅ Best model saved at epoch 7 with validation loss: 0.2550
Epoch 8/20, Train Loss: 0.1603, Val Loss: 0.2218
✅ Best model saved at epoch 8 with validation loss: 0.2218
Epoch 9/20, Train Loss: 0.1552, Val Loss: 0.1919
✅ Best model saved at epoch 9 with validation loss: 0.1919
Epoch 10/20, Train Loss: 0.1

Inference with Batch Greedy Search

In [ ]:
import torch
from transformers import LlamaForCausalLM, LlamaTokenizer
from peft import PeftModel

def generate_answer(questions):
    """
    Generate answers for a batch of questions using greedy search.

    Args:
        questions (list of str): A list of two questions.

    Returns:
        list of str: The list of generated answers.
    """
    model.eval()

    # Create input texts for each question
    input_texts = [f"Question: {q}\nAnswer:" for q in questions]

    # Tokenize the batch with appropriate padding and truncation
    inputs = tokenizer(
        input_texts,
        return_tensors="pt",
        truncation=True,
        padding=True,  # pad to the longest sequence in the batch
        max_length=256
    ).to(model.device)

    print('input_ids:', inputs['input_ids'].shape)

    # Generate outputs using greedy search (do_sample=False ensures greedy decoding)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=50,
            pad_token_id=tokenizer.eos_token_id,
            do_sample=False  # Explicitly disable sampling to use greedy search
        )

    # Decode each generated sequence and extract the answer portion
    answers = []
    for output in outputs:
        decoded_text = tokenizer.decode(output, skip_special_tokens=True)
        # Assuming the output contains the prompt "Answer:" and the generated answer follows it.
        answer = decoded_text.split("Answer:")[-1].strip()
        answers.append(answer)

    return answers


# Path to the best fine-tuned model
best_model_path = "./llama-lora-qa-best"

# Load the LoRA fine-tuned adapter
model = PeftModel.from_pretrained(base_model, best_model_path).to("cuda" if torch.cuda.is_available() else "cpu")

# Load the fine-tuned tokenizer
# tokenizer = LlamaTokenizer.from_pretrained(best_model_path)
tokenizer = AutoTokenizer.from_pretrained(best_model_path, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token  # Ensure correct padding


# Example usage with two sample questions:
questions = [
    "Provide only short answer of the question: Who created the Mona Lisa?",
    "Provide only short answer of the question: What is the capital of France?"
]
answers = generate_answer(questions)
for q, a in zip(questions, answers):
    print(f"Q: {q}\nA: {a}\n")


input_ids: torch.Size([2, 22])


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Q: Provide only short answer of the question: Who created the Mona Lisa?
A: Leonardo da Vinci.

Q: Provide only short answer of the question: What is the capital of France?
A: Paris.



In [ ]:
!pip install --upgrade peft

MoRA

In [ ]:
import torch
import os
import random
import numpy as np
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM, AdamW, BitsAndBytesConfig
from datasets import Dataset

# Import the MoRA adapter classes.
# Note: This example assumes that you have a MoRA implementation in your PEFT library.
from peft import get_peft_model, MoRAConfig, TaskType

# ------------------------------
# 1. Set random seed for reproducibility
# ------------------------------
def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(50)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ------------------------------
# 2. Load LLaMA 2.7B tokenizer and model with 4-bit quantization
# ------------------------------
model_name = "Meta-Llama/Llama-2-7b-chat-hf"

# Load the appropriate tokenizer (using AutoTokenizer for LLaMA-2)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token  # Ensure a valid padding token

# Configure 4-bit quantization using bitsandbytes
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",               # Normalized Float 4 (better than standard FP4)
    bnb_4bit_use_double_quant=True,          # Uses secondary quantization for better precision
    bnb_4bit_compute_dtype=torch.float16     # Keeps computation in FP16 for stability
)

# Load the model with 4-bit quantization
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)
base_model.config.pad_token_id = tokenizer.eos_token_id

# ------------------------------
# 3. Configure MoRA for memory-efficient fine-tuning
# ------------------------------
# (This configuration is analogous to the LoRA configuration but uses MoRA-specific parameters.)
mora_config = MoRAConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,                      # Rank of the update matrices
    mora_alpha=16,            # Scaling factor (similar in spirit to lora_alpha)
    mora_dropout=0.1,         # Dropout probability for the adapter layers
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"]  # The modules to inject MoRA into
)

# Wrap the base model with MoRA adapters
model = get_peft_model(base_model, mora_config)
model.print_trainable_parameters()  # Verify which parameters will be trained

# ------------------------------
# 4. Prepare the training and validation data (QA samples)
# ------------------------------
train_qa_samples = [
    {"question": "What is the capital of France?", "answer": "Paris."},
    {"question": "Who wrote '1984'?", "answer": "George Orwell."},
    {"question": "What is the largest planet?", "answer": "Jupiter."},
    {"question": "Who painted the Mona Lisa?", "answer": "Leonardo da Vinci."},
    {"question": "What is the speed of light?", "answer": "299,792 km/s."}
]

valid_qa_samples = [
    {"question": "Which city is the capital of France?", "answer": "Paris."},
    {"question": "Who is the author of '1984'?", "answer": "George Orwell."},
    {"question": "What planet is the biggest?", "answer": "Jupiter."},
    {"question": "Who created the Mona Lisa?", "answer": "Leonardo da Vinci."},
    {"question": "How fast does light travel?", "answer": "299,792 km/s."}
]

def preprocess_data(example):
    # Combine question and answer into a single prompt
    input_text = f"Question: {example['question']}\nAnswer: {example['answer']}"
    inputs = tokenizer(input_text, truncation=True, padding="max_length", max_length=256)

    # Create labels that are identical to the inputs
    labels = inputs["input_ids"].copy()

    # Mask out (set to -100) the tokens corresponding to the question and any padding
    question_length = len(tokenizer(f"Question: {example['question']}\nAnswer:")["input_ids"]) - 1
    for i in range(len(labels)):
        if i < question_length or labels[i] == tokenizer.pad_token_id:
            labels[i] = -100  # This token will be ignored in the loss computation

    inputs["labels"] = labels
    return inputs

# Convert to Hugging Face datasets and preprocess
dataset_train = Dataset.from_list(train_qa_samples).map(preprocess_data, remove_columns=["question", "answer"])
dataset_valid = Dataset.from_list(valid_qa_samples).map(preprocess_data, remove_columns=["question", "answer"])

def collate_fn(batch):
    input_ids = torch.tensor([item["input_ids"] for item in batch], dtype=torch.long)
    attention_mask = torch.tensor([item["attention_mask"] for item in batch], dtype=torch.long)
    labels = torch.tensor([item["labels"] for item in batch], dtype=torch.long)
    return input_ids, attention_mask, labels

batch_size = 2
train_loader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
valid_loader = DataLoader(dataset_valid, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

# ------------------------------
# 5. Set up optimizer and loss function
# ------------------------------
optimizer = AdamW(model.parameters(), lr=3e-5)
criterion = torch.nn.CrossEntropyLoss(ignore_index=-100)

def save_best_model(model, tokenizer, epoch, best_loss, current_loss, save_path="./llama-mora-qa-best"):
    if current_loss < best_loss:
        best_loss = current_loss
        os.makedirs(save_path, exist_ok=True)
        model.save_pretrained(save_path)
        tokenizer.save_pretrained(save_path)
        print(f"✅ Best model saved at epoch {epoch} with validation loss: {best_loss:.4f}")
    return best_loss

# ------------------------------
# 6. Training and Validation Functions
# ------------------------------
def validate(model, dataloader, criterion):
    model.eval()
    total_loss = 0

    with torch.no_grad():
        for batch in dataloader:
            input_ids, attention_mask, labels = [x.to(device) for x in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            # Shift logits and labels so that predictions align with the targets
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()
            shift_logits = shift_logits.view(-1, shift_logits.size(-1))
            shift_labels = shift_labels.view(-1)

            loss = criterion(shift_logits, shift_labels)
            total_loss += loss.item()

    return total_loss / len(dataloader)

def train(model, train_loader, valid_loader, optimizer, criterion, num_epochs=3):
    best_val_loss = float("inf")

    for epoch in range(num_epochs):
        model.train()
        total_train_loss = 0

        for batch in train_loader:
            input_ids, attention_mask, labels = [x.to(device) for x in batch]
            optimizer.zero_grad()

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()
            shift_logits = shift_logits.view(-1, shift_logits.size(-1))
            shift_labels = shift_labels.view(-1)

            loss = criterion(shift_logits, shift_labels)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()

        avg_train_loss = total_train_loss / len(train_loader)
        avg_val_loss = validate(model, valid_loader, criterion)
        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

        best_val_loss = save_best_model(model, tokenizer, epoch+1, best_val_loss, avg_val_loss)

# ------------------------------
# 7. Start Training
# ------------------------------
train(model, train_loader, valid_loader, optimizer, criterion, num_epochs=20)

# ------------------------------
# 8. Inference: Load the best fine-tuned MoRA adapter and generate answers
# ------------------------------
from peft import PeftModel

best_model_path = "./llama-mora-qa-best"
# Load the base model with the fine-tuned MoRA adapter
model = PeftModel.from_pretrained(base_model, best_model_path).to(device)

# Load the corresponding tokenizer
tokenizer = AutoTokenizer.from_pretrained(best_model_path, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token

def generate_answer(question):
    model.eval()
    input_text = f"Question: {question}\nAnswer:"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=256).to(device)

    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=50, pad_token_id=tokenizer.eos_token_id)

    # Extract the answer part from the generated text
    answer = tokenizer.decode(output[0], skip_special_tokens=True).split("Answer:")[-1].strip()
    return answer

# Example inference
question = "Provide only a short answer to the question: Who created the Mona Lisa?"
answer = generate_answer(question)
print(f"Q: {question}\nA: {answer}")


ImportError: cannot import name 'MoRAConfig' from 'peft' (/usr/local/lib/python3.11/dist-packages/peft/__init__.py)

In [ ]:
import torch
import os
import random
import numpy as np
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM, AdamW, BitsAndBytesConfig
from datasets import Dataset

# Import the MoRA adapter classes.
# Note: This example assumes that you have a MoRA implementation in your PEFT library.
from peft import get_peft_model, TaskType

from peft.tuners.lora import LoraConfig, TaskType

class MoRAConfig(LoraConfig):
    def __init__(
        self,
        task_type: TaskType,
        r: int,
        mora_alpha: int,
        mora_dropout: float,
        target_modules: list,
        **kwargs  # Additional parameters if needed
    ):
        # Initialize the parent class with similar parameters.
        super().__init__(
            task_type=task_type,
            r=r,
            lora_alpha=mora_alpha,       # Reuse the lora_alpha parameter
            lora_dropout=mora_dropout,     # Reuse the lora_dropout parameter
            target_modules=target_modules,
            **kwargs
        )
        # Store MoRA-specific parameters (if different behavior is needed).
        self.mora_alpha = mora_alpha
        self.mora_dropout = mora_dropout
        # Add any additional customizations here.

# Example usage:
# from mora_config import MoRAConfig
# mora_config = MoRAConfig(
#     task_type=TaskType.CAUSAL_LM,
#     r=8,
#     mora_alpha=16,
#     mora_dropout=0.1,
#     target_modules=["q_proj", "k_proj", "v_proj", "o_proj"]
# )


# ------------------------------
# 1. Set random seed for reproducibility
# ------------------------------
def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(50)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ------------------------------
# 2. Load LLaMA 2.7B tokenizer and model with 4-bit quantization
# ------------------------------
model_name = "Meta-Llama/Llama-2-7b-chat-hf"

# Load the appropriate tokenizer (using AutoTokenizer for LLaMA-2)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token  # Ensure a valid padding token

# Configure 4-bit quantization using bitsandbytes
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",               # Normalized Float 4 (better than standard FP4)
    bnb_4bit_use_double_quant=True,          # Uses secondary quantization for better precision
    bnb_4bit_compute_dtype=torch.float16     # Keeps computation in FP16 for stability
)

# Load the model with 4-bit quantization
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)
base_model.config.pad_token_id = tokenizer.eos_token_id

# ------------------------------
# 3. Configure MoRA for memory-efficient fine-tuning
# ------------------------------
# (This configuration is analogous to the LoRA configuration but uses MoRA-specific parameters.)
# from mora_config import MoRAConfig
mora_config = MoRAConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    mora_alpha=16,
    mora_dropout=0.1,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"]
)

# Wrap the base model with MoRA adapters
model = get_peft_model(base_model, mora_config)
model.print_trainable_parameters()  # Verify which parameters will be trained

# ------------------------------
# 4. Prepare the training and validation data (QA samples)
# ------------------------------
train_qa_samples = [
    {"question": "What is the capital of France?", "answer": "Paris."},
    {"question": "Who wrote '1984'?", "answer": "George Orwell."},
    {"question": "What is the largest planet?", "answer": "Jupiter."},
    {"question": "Who painted the Mona Lisa?", "answer": "Leonardo da Vinci."},
    {"question": "What is the speed of light?", "answer": "299,792 km/s."}
]

valid_qa_samples = [
    {"question": "Which city is the capital of France?", "answer": "Paris."},
    {"question": "Who is the author of '1984'?", "answer": "George Orwell."},
    {"question": "What planet is the biggest?", "answer": "Jupiter."},
    {"question": "Who created the Mona Lisa?", "answer": "Leonardo da Vinci."},
    {"question": "How fast does light travel?", "answer": "299,792 km/s."}
]

def preprocess_data(example):
    # Combine question and answer into a single prompt
    input_text = f"Question: {example['question']}\nAnswer: {example['answer']}"
    inputs = tokenizer(input_text, truncation=True, padding="max_length", max_length=256)

    # Create labels that are identical to the inputs
    labels = inputs["input_ids"].copy()

    # Mask out (set to -100) the tokens corresponding to the question and any padding
    question_length = len(tokenizer(f"Question: {example['question']}\nAnswer:")["input_ids"]) - 1
    for i in range(len(labels)):
        if i < question_length or labels[i] == tokenizer.pad_token_id:
            labels[i] = -100  # This token will be ignored in the loss computation

    inputs["labels"] = labels
    return inputs

# Convert to Hugging Face datasets and preprocess
dataset_train = Dataset.from_list(train_qa_samples).map(preprocess_data, remove_columns=["question", "answer"])
dataset_valid = Dataset.from_list(valid_qa_samples).map(preprocess_data, remove_columns=["question", "answer"])

def collate_fn(batch):
    input_ids = torch.tensor([item["input_ids"] for item in batch], dtype=torch.long)
    attention_mask = torch.tensor([item["attention_mask"] for item in batch], dtype=torch.long)
    labels = torch.tensor([item["labels"] for item in batch], dtype=torch.long)
    return input_ids, attention_mask, labels

batch_size = 2
train_loader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
valid_loader = DataLoader(dataset_valid, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

# ------------------------------
# 5. Set up optimizer and loss function
# ------------------------------
optimizer = AdamW(model.parameters(), lr=3e-5)
criterion = torch.nn.CrossEntropyLoss(ignore_index=-100)

def save_best_model(model, tokenizer, epoch, best_loss, current_loss, save_path="./llama-mora-qa-best"):
    if current_loss < best_loss:
        best_loss = current_loss
        os.makedirs(save_path, exist_ok=True)
        model.save_pretrained(save_path)
        tokenizer.save_pretrained(save_path)
        print(f"✅ Best model saved at epoch {epoch} with validation loss: {best_loss:.4f}")
    return best_loss

# ------------------------------
# 6. Training and Validation Functions
# ------------------------------
def validate(model, dataloader, criterion):
    model.eval()
    total_loss = 0

    with torch.no_grad():
        for batch in dataloader:
            input_ids, attention_mask, labels = [x.to(device) for x in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            # Shift logits and labels so that predictions align with the targets
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()
            shift_logits = shift_logits.view(-1, shift_logits.size(-1))
            shift_labels = shift_labels.view(-1)

            loss = criterion(shift_logits, shift_labels)
            total_loss += loss.item()

    return total_loss / len(dataloader)

def train(model, train_loader, valid_loader, optimizer, criterion, num_epochs=3):
    best_val_loss = float("inf")

    for epoch in range(num_epochs):
        model.train()
        total_train_loss = 0

        for batch in train_loader:
            input_ids, attention_mask, labels = [x.to(device) for x in batch]
            optimizer.zero_grad()

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()
            shift_logits = shift_logits.view(-1, shift_logits.size(-1))
            shift_labels = shift_labels.view(-1)

            loss = criterion(shift_logits, shift_labels)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()

        avg_train_loss = total_train_loss / len(train_loader)
        avg_val_loss = validate(model, valid_loader, criterion)
        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

        best_val_loss = save_best_model(model, tokenizer, epoch+1, best_val_loss, avg_val_loss)

# ------------------------------
# 7. Start Training
# ------------------------------
train(model, train_loader, valid_loader, optimizer, criterion, num_epochs=20)

# ------------------------------
# 8. Inference: Load the best fine-tuned MoRA adapter and generate answers
# ------------------------------
from peft import PeftModel

best_model_path = "./llama-mora-qa-best"
# Load the base model with the fine-tuned MoRA adapter
model = PeftModel.from_pretrained(base_model, best_model_path).to(device)

# Load the corresponding tokenizer
tokenizer = AutoTokenizer.from_pretrained(best_model_path, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token

def generate_answer(question):
    model.eval()
    input_text = f"Question: {question}\nAnswer:"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=256).to(device)

    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=50, pad_token_id=tokenizer.eos_token_id)

    # Extract the answer part from the generated text
    answer = tokenizer.decode(output[0], skip_special_tokens=True).split("Answer:")[-1].strip()
    return answer

# Example inference
question = "Provide only a short answer to the question: Who created the Mona Lisa?"
answer = generate_answer(question)
print(f"Q: {question}\nA: {answer}")


ImportError: cannot import name 'TaskType' from 'peft.tuners.lora' (/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/__init__.py)

In [ ]:
!

In [ ]:
from peft import get_peft_model, MoRAConfig, TaskType
# (This configuration is analogous to the LoRA configuration but uses MoRA-specific parameters.)
mora_config = MoRAConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,                      # Rank of the update matrices
    mora_alpha=16,            # Scaling factor (similar in spirit to lora_alpha)
    mora_dropout=0.1,         # Dropout probability for the adapter layers
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"]  # The modules to inject MoRA into
)

# Wrap the base model with MoRA adapters
model = get_peft_model(base_model, mora_config)
model.print_trainable_parameters()  # Verify which parameters will be trained